In [ ]:
%pip install beautifulSoup4 requests
# 라이브러리를 같이 설치하게 되면 서로 의존관계를 파악해서 맞는 버전으로 설치해줌
#%pip install beautifulSoup4  / #%pip install requests  ==> 별도 설치보다 같이 설치하는게 좋음.

Note: you may need to restart the kernel to use updated packages.


In [9]:
# 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [ ]:
# 크롤링 가능 여부 확인
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.naver.com/robots.txt")
rp.read()
url = 'https://news.naver.com/'
rp.can_fetch('*', url)

# False로 나오면 크롤링 불가능.. 

False

In [ ]:
# 크롤링 가능 여부 확인 : ex. 할리스 매장정보 크롤링 가능 True로 크롤링 가능함
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.hollys.co.kr/robots.txt")
rp.read()
url = 'https://www.hollys.co.kr/store/korea/korStore2.do'
rp.can_fetch('*', url)

True

In [ ]:
# 위키피디아 미국 ETF 웹 페이지에서 필요한 정보를 스크래핑하여 딕셔너리 형태로 변수 etfs에 저장
url = "https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds"

# 크롤링 정책중에 User-Agent_Policy 가 있으면, header정보 추가 가능
# 크롤링 가능 여부는, 사이트 주소 뒤에 robots.txt 하면 ueser-agent 정보가 나옴. 거기 disallow 또는 allow로 정보 나옴. 
# https://foundation.wikimedia.org/wiki/Policy:Wikimedia_Foundation_User-Agent_Policy 
# 상기 사이트에서 user agent 정보 가져오기
header = {
"User-Agent": "CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org) generic-library/0.0"
}

resp = requests.get(url, headers=header )
soup = BeautifulSoup(resp.text, 'lxml')   
#F12해서 기존 div > ul > li  위치 확인해서 copy selector로 가져오기 
rows = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > ul:nth-child(11) > li') 

for row in rows:
    print(row.text)

iShares Core S&P Total US Stock Mkt (NYSE Arca: ITOT)
iShares MSCI ACWI Index (Nasdaq: ACWI)
iShares Russell 3000 Index (NYSE Arca: IWV)
Schwab US Broad Market ETF (NYSE Arca: SCHB)
Schwab Fundamental U.S. Broad Market Index ETF (NYSE Arca: FNDB)
Vanguard Total World Stock (NYSE Arca: VT), tracks the FTSE All-World Index
Vanguard Total Stock Market (NYSE Arca: VTI), tracks the MSCI US Broad Market Index
Vanguard Total International Stock (NYSE Arca: VXUS), tracks the MSCI All Country World ex-USA Investable Market Index
Vanguard Russell 3000 (NYSE Arca: VTHR), tracks 98% of the US market


In [11]:
etfs = {}
for row in rows:
    
    try:
        etf_name = re.findall("(.+?)\s\(", row.text)
        etf_market = re.findall("\((.+?):", row.text)
        etf_ticker = re.findall(":\s(.+)\)", row.text)
        
        if (len(etf_ticker) > 0) & (len(etf_market) > 0) & (len(etf_name) > 0):
            etfs[etf_ticker[0]] = [etf_market[0], etf_name[0]]

    except AttributeError as err:
        pass    

# etfs 딕셔너리 출력
print(etfs)

{}


<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\('
<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\('
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\playdata2\AppData\Local\Temp\ipykernel_13292\1232222225.py:5: SyntaxWarning: invalid escape sequence '\s'
  etf_name = re.findall("(.+?)\s\(", row.text)
C:\Users\playdata2\AppData\Local\Temp\ipykernel_13292\1232222225.py:6: SyntaxWarning: invalid escape sequence '\('
  etf_market = re.findall("\((.+?):", row.text)
C:\Users\playdata2\AppData\Local\Temp\ipykernel_13292\1232222225.py:7: SyntaxWarning: invalid escape sequence '\s'
  etf_ticker = re.findall(":\s(.+)\)", row.text)


In [12]:
# etfs 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame(etfs)
df

""
